In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyproj import Transformer
print("Hello World!")


Hello World!


In [2]:
# Read 311_dict.xlsx to df2
df2 = pd.read_excel('311_dict.xlsx')

# Set display options to show all columns and rows and maximum column width
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
df2

,Field Name,Description,Data Type,Comment
0,_id,"The number for the respective case on the spreadsheet, automatically assigned by the export.",Text,NaN
1,Category,"This general category was developed to place 311 services in a high level category, \nDifferent than their respective department.",Text,"There are eleven categories: \n'Animals', 'Historic Preservation', 'Streets & Infrastructure','Dockless Vehicle','Traffic Signals and Signs', 'Information','Property Maintenance', 'Graffiti', 'Parks', 'Health & Sanitation','Solid Waste Services'"
2,CASEID,The unique case reference number is assigned by the 311 Lagan customer relationship \nManagement system.,10 numeric characters,NaN
3,OPENEDDATETIME,The date and time that a case was submitted.,mm/dd/yyyy hh:mm:ss AM/PM,NaN
4,SLA_Date,"Every service request type has a due date assigned to the request, based on the request type name. \nThe SLA Date is the due date and time for the request type based on the service level agreement (SLA). \nEach service request Type has a timeframe in which it is scheduled to be addressed.",mm/dd/yyyy hh:mm:ss AM/PM,NaN
5,CLOSEDDATETIME,"The date and time that the case/request was was closed. If blank, the request \nHas not been closed as of the Report Ending Date.",mm/dd/yyyy hh:mm:ss AM/PM,"If there is no closed date, the case is still open as of the Reporting \nEnding Date."
6,Late (Yes/No),This indicates whether the case has surpassed its Service Level Agreement due date for \nthe specific service request.,YES/NO,NaN
7,Dept,The City department to whom the case is assigned.,Text,"There are Nine Departments:\n 'Animal Care Services', 'Office of Historic Preservation', 'Public Works', '311', 'Development Services', \n'Parks and Recreation', 'Human Services', 'Solid Waste Management', 'Metro Health'"
8,REASONNAME,The department division within the City department to whom the case is assigned.,Text,"Department Division within City Department: \n'Signals', 'Waste Collection', 'Traffic Engineering Design', 'Signs and Markings', 'Storm Water Engineering',\n'Field Operations', 'Historic Preservation', 'Traffic Operations','Trades', 'Customer Service', 'Streets', 'Code Enforcement','Land Development', 'Stormwater', 'Homeless Services', 'Natural Resources', 'Solid Waste', 'Park Projects', 'Urban Forestry', 'Miscellaneous', 'Food Establishments','Clean and Green', \n'Facility License', 'Vector','General Sanitation', 'Brush'"
9,TYPENAME,"The service request type name for the issue being reported. Examples include stray animals, \nPotholes, overgrown yards, junk vehicles, traffic signal malfunctions, etc.",Text,NaN


In [15]:
# Define the path to the CSV file
#Laptop
# csv_file_path = r'C:\Users\mrwoo\OneDrive\Desktop\SA_Interview\allservicecalls.csv'

#Server
csv_file_path = "/home/mrwoo/Documents/all_331_transformed.csv"

# Reset display options to default
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Display the DataFrame
df

,Category,CASEID,OPENEDDATETIME,SLA_Date,CLOSEDDATETIME,Late (Yes/No),Dept,REASONNAME,TYPENAME,CaseStatus,SourceID,OBJECTDESC,Council District,Report Starting Date,Report Ending Date,LATITUDE,LONGITUDE
0,Traffic Signals and Signs,1014001765,11/3/2017,9/22/2021,2023-03-16,YES,Public Works,Signals,Signal Timing Modification By Engineer,Closed,Constituent Call,PATRON and POTEET JDTN FY,4,3/2/2023,3/2/2024,29.338427,-98.550563
1,Solid Waste Services,1014303843,3/7/2018,3/16/2018,2023-03-08,YES,Solid Waste Management,Waste Collection,Additional Cart Request,Closed,Constituent Call,"1819 POPLAR ST W, San Antonio, 78207",1,3/2/2023,3/2/2024,29.440466,-98.519462
2,Traffic Signals and Signs,1014504120,5/11/2018,10/31/2018,2023-03-16,YES,Public Works,Traffic Engineering Design,Traffic Signal New Request,Closed,Constituent Call,HILDEBRAND E and NEW BRNFLS N,2,3/2/2023,3/2/2024,29.465814,-98.460859
3,Traffic Signals and Signs,1014549690,5/27/2018,11/15/2018,2023-03-16,YES,Public Works,Traffic Engineering Design,Traffic Signal New Request,Closed,Constituent Call,PERRIN BEITEL and SUNSHADOW ST,10,3/2/2023,3/2/2024,29.523482,-98.410947
4,Solid Waste Services,1014569491,6/3/2018,6/11/2018,NaN,YES,Solid Waste Management,Waste Collection,Damaged Cart,Open,Constituent Call,"220 BROADWAY, San Antonio, 78205",1,3/2/2023,3/2/2024,29.428094,-98.487158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606056,Animals,1019442050,3/1/2024,3/11/2024,NaN,NO,Animal Care Services,Field Operations,Animals(Public Nuisance),Open,Constituent Call,"319 AMBERDALE OAK, SAN ANTONIO, 78249",8,3/2/2023,3/2/2024,29.575311,-98.629121
606057,Animals,1019442051,3/1/2024,3/11/2024,NaN,NO,Animal Care Services,Field Operations,Animals(Public Nuisance),Open,Constituent Call,"11631 HIDDEN TERRACE, UBC, 78245",0,3/2/2023,3/2/2024,29.409398,-98.723522
606058,Animals,1019442052,3/1/2024,3/2/2024,2024-03-01,NO,Animal Care Services,Field Operations,Third Shift,Closed,Constituent Call,"55 CONSUELO, SAN ANTONIO, 78228",5,3/2/2023,3/2/2024,29.441727,-98.582815
606059,Animals,1019442053,3/1/2024,3/2/2024,NaN,NO,Animal Care Services,Field Operations,Third Shift,Open,Constituent Call,"7950 MEADOW STAR, SAN ANTONIO, 78227",6,3/2/2023,3/2/2024,29.420643,-98.644207


## The Below Code was used to transform the NAD(1984) Coordinate system to latitude and longitude


In [16]:

# Define the projection for State Plane Texas South Central FIPS 4204 Feet
state_plane_proj = Transformer.from_crs('epsg:2278', 'epsg:4326', always_xy=True)

# Function to perform the coordinate transformation
def convert_coordinates(xcoord, ycoord):
    lon, lat = state_plane_proj.transform(xcoord, ycoord)
    return lat, lon

# Apply the transformation function to the 'XCOORD' and 'YCOORD' columns
df['LATITUDE'], df['LONGITUDE'] = zip(*df.apply(lambda row: convert_coordinates(row['XCOORD'], row['YCOORD']), axis=1))

# Drop the original 'XCOORD' and 'YCOORD' columns if you don't need them anymore
df.drop(columns=['XCOORD', 'YCOORD'], inplace=True)
df

In [39]:
# #Convert df into .csv file
# file_path = r"C:\Users\mrwoo\OneDrive\Desktop\SA_Interview\all_331_transformed.csv"
# df.to_csv(file_path, index=False)


In [37]:
# # Remove duplicates based on 'CASEID' column
# df = df.drop_duplicates(subset='CASEID')

# # Check the shape of the DataFrame after removing duplicates
# print("Shape of DataFrame after removing duplicates:", df.shape)

# # verify the unique count of 'CASEID' after removing duplicates
# unique_case_ids = df['CASEID'].nunique()
# print("Unique count of CASEID after removing duplicates:", unique_case_ids)
# # NO DUPLICATES
# # Shape of DataFrame after removing duplicates: (606061, 17)
# # Unique count of CASEID after removing duplicates: 606061

Shape of DataFrame after removing duplicates: (606061, 17)
Unique count of CASEID after removing duplicates: 606061


In [19]:
df

,Category,CASEID,OPENEDDATETIME,SLA_Date,CLOSEDDATETIME,Late (Yes/No),Dept,REASONNAME,TYPENAME,CaseStatus,SourceID,OBJECTDESC,Council District,Report Starting Date,Report Ending Date,LATITUDE,LONGITUDE
0,Traffic Signals and Signs,1014001765,11/3/2017,9/22/2021,2023-03-16,YES,Public Works,Signals,Signal Timing Modification By Engineer,Closed,Constituent Call,PATRON and POTEET JDTN FY,4,3/2/2023,3/2/2024,29.338427,-98.550563
1,Solid Waste Services,1014303843,3/7/2018,3/16/2018,2023-03-08,YES,Solid Waste Management,Waste Collection,Additional Cart Request,Closed,Constituent Call,"1819 POPLAR ST W, San Antonio, 78207",1,3/2/2023,3/2/2024,29.440466,-98.519462
2,Traffic Signals and Signs,1014504120,5/11/2018,10/31/2018,2023-03-16,YES,Public Works,Traffic Engineering Design,Traffic Signal New Request,Closed,Constituent Call,HILDEBRAND E and NEW BRNFLS N,2,3/2/2023,3/2/2024,29.465814,-98.460859
3,Traffic Signals and Signs,1014549690,5/27/2018,11/15/2018,2023-03-16,YES,Public Works,Traffic Engineering Design,Traffic Signal New Request,Closed,Constituent Call,PERRIN BEITEL and SUNSHADOW ST,10,3/2/2023,3/2/2024,29.523482,-98.410947
4,Solid Waste Services,1014569491,6/3/2018,6/11/2018,NaN,YES,Solid Waste Management,Waste Collection,Damaged Cart,Open,Constituent Call,"220 BROADWAY, San Antonio, 78205",1,3/2/2023,3/2/2024,29.428094,-98.487158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606056,Animals,1019442050,3/1/2024,3/11/2024,NaN,NO,Animal Care Services,Field Operations,Animals(Public Nuisance),Open,Constituent Call,"319 AMBERDALE OAK, SAN ANTONIO, 78249",8,3/2/2023,3/2/2024,29.575311,-98.629121
606057,Animals,1019442051,3/1/2024,3/11/2024,NaN,NO,Animal Care Services,Field Operations,Animals(Public Nuisance),Open,Constituent Call,"11631 HIDDEN TERRACE, UBC, 78245",0,3/2/2023,3/2/2024,29.409398,-98.723522
606058,Animals,1019442052,3/1/2024,3/2/2024,2024-03-01,NO,Animal Care Services,Field Operations,Third Shift,Closed,Constituent Call,"55 CONSUELO, SAN ANTONIO, 78228",5,3/2/2023,3/2/2024,29.441727,-98.582815
606059,Animals,1019442053,3/1/2024,3/2/2024,NaN,NO,Animal Care Services,Field Operations,Third Shift,Open,Constituent Call,"7950 MEADOW STAR, SAN ANTONIO, 78227",6,3/2/2023,3/2/2024,29.420643,-98.644207


# Converting 'CLOSEDDATETIME' column to 00/00/0000

In [23]:
# Convert 'CLOSEDDATETIME' column to datetime format
df['CLOSEDDATETIME'] = pd.to_datetime(df['CLOSEDDATETIME'])

# Convert datetime objects to strings with the desired format
df['CLOSEDDATETIME'] = df['CLOSEDDATETIME'].dt.strftime('%m/%d/%Y')

In [16]:
# Row Count Jan 2020+
percentage_lost = (606061 - row_count)/606061
print(row_count)
print(percentage_lost)

548841
0.0944129386315899


In [27]:
# # Convert 'CLOSEDDATETIME' column to datetime format
# df['CLOSEDDATETIME'] = pd.to_datetime(df['CLOSEDDATETIME'], format='%m/%d/%Y')

# # Filter DataFrame for dates after January 2022
# date_df = df[df['CLOSEDDATETIME'] > pd.Timestamp('01/01/2016')]

# # Get rows that are being filtered out
# filtered_out_rows = df[~df.index.isin(date_df.index)]

# # Display the filtered out rows
# filtered_out_rows

,Category,CASEID,OPENEDDATETIME,SLA_Date,CLOSEDDATETIME,Late (Yes/No),Dept,REASONNAME,TYPENAME,CaseStatus,SourceID,OBJECTDESC,Council District,Report Starting Date,Report Ending Date,LATITUDE,LONGITUDE
4,Solid Waste Services,1014569491,6/3/2018,6/11/2018,NaT,YES,Solid Waste Management,Waste Collection,Damaged Cart,Open,Constituent Call,"220 BROADWAY, San Antonio, 78205",1,3/2/2023,3/2/2024,29.428094,-98.487158
26,Animals,1015248998,2/2/2019,2/16/2019,NaT,YES,Animal Care Services,Field Operations,Animal Permits Request,Open,Constituent Call,"2033 FRIO CITY RD, San Antonio, 78226",5,3/2/2023,3/2/2024,29.390961,-98.541106
27,Animals,1015260515,2/6/2019,2/20/2019,NaT,YES,Animal Care Services,Field Operations,Animal Permits Request,Open,Constituent Call,"2018 FITCH ST, San Antonio, 78211",5,3/2/2023,3/2/2024,29.370346,-98.535926
35,Animals,1015326342,3/4/2019,3/18/2019,NaT,YES,Animal Care Services,Field Operations,Animal Permits Request,Open,Constituent Call,"117 LORETTA, San Antonio, 78210",5,3/2/2023,3/2/2024,29.397271,-98.484690
38,Animals,1015352127,3/13/2019,3/27/2019,NaT,YES,Animal Care Services,Field Operations,Animal Permits Request,Open,Constituent Call,"4032 E SOUTHCROSS, San Antonio, 78222",3,3/2/2023,3/2/2024,29.374291,-98.415585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606055,Animals,1019442049,3/1/2024,3/11/2024,NaT,NO,Animal Care Services,Field Operations,Animals(Public Nuisance),Open,311 Mobile App,"1003 BURLESON ST, SAN ANTONIO, 78202",2,3/2/2023,3/2/2024,29.433034,-98.465571
606056,Animals,1019442050,3/1/2024,3/11/2024,NaT,NO,Animal Care Services,Field Operations,Animals(Public Nuisance),Open,Constituent Call,"319 AMBERDALE OAK, SAN ANTONIO, 78249",8,3/2/2023,3/2/2024,29.575311,-98.629121
606057,Animals,1019442051,3/1/2024,3/11/2024,NaT,NO,Animal Care Services,Field Operations,Animals(Public Nuisance),Open,Constituent Call,"11631 HIDDEN TERRACE, UBC, 78245",0,3/2/2023,3/2/2024,29.409398,-98.723522
606059,Animals,1019442053,3/1/2024,3/2/2024,NaT,NO,Animal Care Services,Field Operations,Third Shift,Open,Constituent Call,"7950 MEADOW STAR, SAN ANTONIO, 78227",6,3/2/2023,3/2/2024,29.420643,-98.644207
